In [ ]:
import tensorflow as tf
print(tf.test.gpu_device_name())
print("TensorFlow version:", tf.__version__)
!nvidia-smi

Daten Importieren

In [1]:
import os
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split


# Erstellen Sie einen leeren DataFrame, um die kombinierten Daten aufzunehmen
combined_data = pd.DataFrame(columns=['Bildname', 'Porositaet'])

# Verzeichnispfad, in dem Ihre kleinen Excel-Dateien gespeichert sind
xlsx_path = 'Bildervorbearbeitung/Bilder/Auswertung/xlsx'

# Durchlaufen Sie die Dateien im Verzeichnis
for filename in os.listdir(xlsx_path):
    if filename.endswith('.xlsx'):  # Stellen Sie sicher, dass es sich um Excel-Dateien handelt
        # Vollständigen Pfad zur aktuellen Datei erstellen
        file_path = os.path.join(xlsx_path, filename)
        
        # Laden Sie die Daten aus der aktuellen Datei in einen DataFrame
        data = pd.read_excel(file_path)
        
        # Fügen Sie die Daten zum kombinierten DataFrame hinzu
        combined_data = combined_data.append(data, ignore_index=True)

# Speichern Sie die kombinierten Daten in einer neuen Excel-Datei oder verwenden Sie sie wie gewünscht
combined_data.to_excel('import_data.xlsx', index=False)

img_folder_path = "Bildervorbearbeitung/Bilder/Auswertung/Zugeschnittene_Bilder"

X = []
y = []

import_data = pd.read_excel('import_data.xlsx')
for index, row in import_data.iterrows():
    img_name = row['Bildname']
    img_path = os.path.join(img_folder_path, img_name)
    img = cv2.imread(img_path)
    X.append(img)
    y.append(row['Porositaet'])

# Das Array hat die Form (Anzahl_Bilder, 500, 500, 3)
X = np.array(X)  
y = np.array(y)

# Aufteilung in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# test_size gibt den Anteil der Testdaten an (hier 20%)
# random_state ist eine Zufallszahl für die Reproduzierbarkeit der Aufteilung



/var/folders/ny/0_rl0ybx6dn5qlc0nwtzw8s00000gn/T/ipykernel_2106/576617423.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(data, ignore_index=True)


In [2]:
IMG_CHANNELS = 3

def read_and_decode(filename, reshape_dims):
    # Read an image file to a tensor as a sequence of bytes
    image = tf.io.read_file(filename)
    # Convert the tensor to a 3D uint8 tensor
    image = tf.image.decode_jpeg(image, channels=IMG_CHANNELS)
    # Convert 3D uint8 tensor with values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Resize the image to the desired size
    return tf.image.resize(image, reshape_dims)

In [2]:
import numpy as np
X_mean = np.mean(X_train)
X_std = np.std(X_train)

def standardize(x):
    return (x - X_mean) / X_std

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D, Reshape
from keras.layers import Dense, Dropout, Flatten
from keras.layers import BatchNormalization
from numpy import reshape

model = Sequential()

model.add(Lambda(standardize, input_shape=(500, 500, 3)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", input_shape=(500, 500, 3)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(512, activation="relu"))
model.add(Dense(1, activation="linear"))  # 1 output neuron for regression

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae"])  # Using MSE loss for regression

#model.summary()


2023-11-23 08:43:35.443294: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import tensorflow as tf
import keras
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [6]:
model.fit(X_train, y_train, epochs=30)

Epoch 1/30
11/51 [=====>........................] - ETA: 22:41 - loss: 381106.8438 - root_mean_squared_error: 617.3385

In [14]:
model.evaluate(X_test, y_test, verbose=2)

KeyboardInterrupt: 